# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [45]:
# Dependencies and Setup
!jupyter nbextension enable --py gmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [46]:
#importing info from cities.csv
ctyInfo = pd.read_csv('../output_data/cities.csv') 
#making the info into a Data Frame
df = pd.DataFrame(ctyInfo)
df.head()


,City Name,Temperature,Maximum Temp,Wind,Cloudiness,Latitude,Longitude,Humidity
0,Te Anau,9.56,9.56,0.19,39,-45.4167,167.7167,83
1,Novikovo,-10.45,-10.45,3.19,0,51.1537,37.8879,71
2,Rozivka,-5.45,-5.45,4.34,0,46.9385,35.7724,40
3,Kapaa,22.57,23.33,4.12,75,22.0752,-159.3190,88
4,Maceió,27.00,27.00,3.60,0,-9.6658,-35.7353,78


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
#I made a heat map
locations = df[['Latitude', 'Longitude']]
fig = gmaps.figure()
layer = gmaps.heatmap_layer(locations, weights=df['Humidity'], dissipating=False, max_intensity= 100, point_radius= 3)
fig.add_layer(layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
#I like a little bit of couds so I made the coul
ndf = df.loc[((df['Humidity']< 80) & (df['Humidity'] > 70) & (df['Wind']< 10) & (df['Cloudiness']< 10)& (df['Temperature'] > 10))]
ndf.head()

,City Name,Temperature,Maximum Temp,Wind,Cloudiness,Latitude,Longitude,Humidity
4,Maceió,27.00,27.00,3.60,0,-9.6658,-35.7353,78
42,Kaitangata,10.53,10.53,2.51,1,-46.2817,169.8464,77
109,São João da Barra,27.00,27.00,5.14,0,-21.6403,-41.0511,78
159,Bandarbeyla,24.97,24.97,3.57,0,9.4942,50.8122,74
237,Fortuna,11.91,13.33,5.66,1,40.5982,-124.1573,71


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [74]:
hotel_df = ndf
hotel_df.reset_index()
hotel_df["Hotel Name"] = ""


    
for index, row in hotel_df.iterrows():
    try:
        target_city = row['City Name']
        lat = row['Latitude']
        lng = row['Longitude']
        target_url = (f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&keyword=Lodging&radius=5000&type=Lodging&key={g_key}')
        #print(target_url)
        request = requests.get(target_url).json()
        hotel_df.loc[index, "Hotel Name"] = request['results'][0]['name']
    except:
        print()


<ipython-input-74-d793fca55800>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


In [80]:
pprint(request['results'])
hotel_df.head()

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 11.6821956, 'lng': 92.7238641},
               'viewport': {'northeast': {'lat': 11.68356012989272,
                                          'lng': 92.72526322989272},
                            'southwest': {'lat': 11.68086047010728,
                                          'lng': 92.72256357010728}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
  'name': 'Salt life',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 945,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/100300813144160516257">A '
                                    'Google User</a>'],
              'photo_reference': 'ATtYBwK_mHuRMrqnOn2PYCza0Kp-jel57ZD0oHaO7eUwQ_aQzQLPfwjx7xKU3-j2-D_4YnDHeChog59zCzmGnGXRUPw00AWM3wtFvZbDWEw9P_CLYmAHRxg8-bdbVVK5qqf-1u-HbFjiMCYYdPlW0cJKopjbJQ0ddWlZrA4INgPo6riaVjIY',
              'width

,City Name,Temperature,Maximum Temp,Wind,Cloudiness,Latitude,Longitude,Humidity,Hotel Name
4,Maceió,27.00,27.00,3.60,0,-9.6658,-35.7353,78,Pousada das Araras
42,Kaitangata,10.53,10.53,2.51,1,-46.2817,169.8464,77,Kaitangata Motor Camp
109,São João da Barra,27.00,27.00,5.14,0,-21.6403,-41.0511,78,POUSADA SOBRE AS ÁGUAS
159,Bandarbeyla,24.97,24.97,3.57,0,9.4942,50.8122,74,JABIR HOTEL
237,Fortuna,11.91,13.33,5.66,1,40.5982,-124.1573,71,THE REDWOOD RIVERWALK HOTEL


In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City Name}</dd>
</dl>
"""
#<dt>Country</dt><dd>{Country}</dd>
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [86]:
#I created the second heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=hotel_df['Humidity'], 
                                 dissipating=False, max_intensity=100,
# I added the marker layer ontop of the heat map                                 point_radius = 1)
marker_layer = gmaps.marker_layer(locations)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))